# Initialize

In [87]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm
from joblib import Parallel, delayed

import matplotlib

from sklearn.preprocessing import OneHotEncoder



# Data processing

In [88]:
# Read the dataset
unprocessed_data_2020_2021_fa_com = pd.read_csv(
    os.getcwd() + "/data/unprocessed_data_2020_2021_fa_com.csv")
unprocessed_data_2019_efl_fa_com = pd.read_csv(
    os.getcwd() + "/data/unprocessed_data_2019_efl_fa_com.csv")
unprocessed_data_2018_efl_fa_com = pd.read_csv(
    os.getcwd() + "/data/unprocessed_data_2018_efl_fa_com.csv")
unprocessed_data_2017_efl_fa_com = pd.read_csv(
    os.getcwd() + "/data/unprocessed_data_2017_efl_fa_com.csv")
unprocessed_data_2017_prem_champ = pd.read_csv(
    os.getcwd() + "/data/unprocessed_data_2017_prem_champ.csv")
unprocessed_data_2019_2020_allsv = pd.read_csv(
    os.getcwd() + "/data/unprocessed_data_2019_2020_allsv.csv")
unprocessed_data_2017_allsv = pd.read_csv(os.getcwd() +
                                          "/data/unprocessed_data_2017_allsv.csv")
unprocessed_data_2018_allsv = pd.read_csv(os.getcwd() +
                                          "/data/unprocessed_data_2018_allsv.csv")
unprocessed_data_2018_prem_champ = pd.read_csv(
    os.getcwd() + "/data/unprocessed_data_2018_prem_champ.csv")
unprocessed_data_2019_prem_champ = pd.read_csv(
    os.getcwd() + "/data/unprocessed_data_2019_prem_champ.csv")
unprocessed_data_2020_prem_champ = pd.read_csv(
    os.getcwd() + "/data/unprocessed_data_2020_prem_champ.csv")

combined_data = pd.concat([
    unprocessed_data_2020_2021_fa_com, unprocessed_data_2019_efl_fa_com,
    unprocessed_data_2018_efl_fa_com, unprocessed_data_2017_efl_fa_com,
    unprocessed_data_2017_prem_champ, unprocessed_data_2019_2020_allsv,
    unprocessed_data_2017_allsv, unprocessed_data_2018_allsv,
    unprocessed_data_2018_prem_champ, unprocessed_data_2019_prem_champ,
    unprocessed_data_2020_prem_champ
],
                          axis=0)

print(combined_data.shape)

# READ IN THE REMAINING DATASETS

(9158, 232)


In [89]:
# Print dataset columns
for x in [0,50,100,150,200]:
    print(combined_data.columns[x:x+50])

Index(['id', 'date', 'status', 'league_id', 'league_name', 'home_team_id',
       'home_team_name', 'away_team_id', 'away_team_name', 'goals_home',
       'goals_away', 'ht_id', 'ht_name', 'at_id', 'at_name', 'fixture',
       'ht_Shots_on_Goal', 'ht_Shots_off_Goal', 'ht_Total_Shots',
       'ht_Blocked_Shots', 'ht_Shots_insidebox', 'ht_Shots_outsidebox',
       'ht_Fouls', 'ht_Corner_Kicks', 'ht_Offsides', 'ht_Ball_Possession',
       'ht_Yellow_Cards', 'ht_Red_Cards', 'ht_Goalkeeper_Saves',
       'ht_Total_passes', 'ht_Passes_accurate', 'ht_Passes_%',
       'at_Shots_on_Goal', 'at_Shots_off_Goal', 'at_Total_Shots',
       'at_Blocked_Shots', 'at_Shots_insidebox', 'at_Shots_outsidebox',
       'at_Fouls', 'at_Corner_Kicks', 'at_Offsides', 'at_Ball_Possession',
       'at_Yellow_Cards', 'at_Red_Cards', 'at_Goalkeeper_Saves',
       'at_Total_passes', 'at_Passes_accurate', 'at_Passes_%', 'index',
       'league_id_home'],
      dtype='object')
Index(['league_season_home', 'date_home',

## Target

In [90]:
# Match outcome
combined_data["result"] = ""
combined_data["result"].loc[
    combined_data["goals_home"] > combined_data["goals_away"]] = "1"
combined_data["result"].loc[
    combined_data["goals_away"] > combined_data["goals_home"]] = "2"
combined_data["result"].loc[combined_data["goals_away"] ==
                            combined_data["goals_home"]] = "x"

# Model target
combined_data["target"] = np.nan
combined_data["target"].loc[combined_data["result"] == "1"] = 0
combined_data["target"].loc[combined_data["result"] == "x"] = 1
combined_data["target"].loc[combined_data["result"] == "2"] = 2

/Users/august/opt/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/august/opt/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/august/opt/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

## One-hot encoding

In [91]:
def dummies_lineup(series, name):
    """One hot encodes series and return df."""
    # specs
    enc = OneHotEncoder(handle_unknown='ignore', drop='first')

    # Fit
    enc.fit(series.values.reshape(-1, 1))

    # Create
    dummies = enc.transform(series.values.reshape(-1, 1)).toarray()
    dummies_name = enc.get_feature_names_out([name])
    df = pd.DataFrame(data=dummies, columns=dummies_name)

    return df

In [92]:
# Create variables that require one-hot
combined_data["dayofweek"] = pd.to_datetime(combined_data["date"]).dt.dayofweek
combined_data["month"] = pd.to_datetime(combined_data["date"]).dt.month
combined_data["hour"] = pd.to_datetime(combined_data["date"]).dt.hour

# Lineups
lineups_home = combined_data["lineups_home"].apply(
    lambda x: x[1:-1].replace("'", "").split(','))
lineups_away = combined_data["lineups_away"].apply(
    lambda x: x[1:-1].replace("'", "").split(','))

lineups_home_0 = lineups_home.str[0].str.split("_").str[0].str.strip()
lineups_home_1 = lineups_home.str[1].str.split("_").str[0].str.strip()
lineups_home_2 = lineups_home.str[2].str.split("_").str[0].str.strip()
lineups_away_0 = lineups_away.str[0].str.split("_").str[0].str.strip()
lineups_away_1 = lineups_away.str[1].str.split("_").str[0].str.strip()
lineups_away_2 = lineups_away.str[2].str.split("_").str[0].str.strip()

# Lineups one-hot
lineups_home_0_df = dummies_lineup(lineups_home_0, "lineups_home_r0")
lineups_home_1_df = dummies_lineup(lineups_home_1, "lineups_home_r1")
lineups_home_2_df = dummies_lineup(lineups_home_2, "lineups_home_r2")
lineups_away_0_df = dummies_lineup(lineups_away_0, "lineups_away_r0")
lineups_away_1_df = dummies_lineup(lineups_away_1, "lineups_away_r1")
lineups_away_2_df = dummies_lineup(lineups_away_2, "lineups_away_r2")

# league name
league_df = dummies_lineup(combined_data["league_name"], "league_name")

# Month
month_df = dummies_lineup(combined_data["month"], "month")

# Weekday
weekday_df = dummies_lineup(combined_data["dayofweek"], "dayofweek")

# Hour
hour_df = dummies_lineup(combined_data["hour"], "hour")

# Combine
one_hot_data = pd.concat([
    lineups_home_0_df, lineups_home_1_df, lineups_home_2_df, lineups_away_0_df,
    lineups_away_1_df, lineups_away_2_df, league_df, month_df, weekday_df,
    hour_df
],
                         axis=1)

## Make numerical

In [93]:
def transform_perc(df, column):
    """Replace object percentage to float."""
    try:
        df[column] = df[column].str.replace("%", "").astype(float).divide(100)
    except AttributeError:
        print("Not a string!")
    return df


perc_columns = [
    "ht_Ball_Possession", "at_Ball_Possession", 'ht_Passes_%', 'at_Passes_%',
    'goals_for_minute_0-15_percentage_home',
    'goals_for_minute_16-30_percentage_home',
    'goals_for_minute_31-45_percentage_home',
    'goals_for_minute_46-60_percentage_home',
    'goals_for_minute_61-75_percentage_home',
    'goals_for_minute_76-90_percentage_home',
    'goals_for_minute_91-105_percentage_home',
    'goals_for_minute_106-120_percentage_home',
    'penalty_scored_percentage_home', 'penalty_missed_percentage_home',
    'cards_yellow_0-15_percentage_home', 'cards_yellow_16-30_percentage_home',
    'cards_yellow_31-45_percentage_home', 'cards_yellow_46-60_percentage_home',
    'cards_yellow_61-75_percentage_home', 'cards_yellow_76-90_percentage_home',
    'cards_yellow_91-105_percentage_home',
    'cards_yellow_106-120_percentage_home', 'cards_red_0-15_percentage_home',
    'cards_red_16-30_percentage_home', 'cards_red_31-45_percentage_home',
    'cards_red_46-60_percentage_home', 'cards_red_61-75_percentage_home',
    'cards_red_76-90_percentage_home', 'cards_red_91-105_percentage_home',
    'cards_red_106-120_percentage_home',
    'goals_for_minute_0-15_percentage_away',
    'goals_for_minute_16-30_percentage_away',
    'goals_for_minute_31-45_percentage_away',
    'goals_for_minute_46-60_percentage_away',
    'goals_for_minute_61-75_percentage_away',
    'goals_for_minute_76-90_percentage_away',
    'goals_for_minute_91-105_percentage_away',
    'goals_for_minute_106-120_percentage_away',
    'penalty_scored_percentage_away', 'penalty_missed_percentage_away',
    'cards_yellow_0-15_percentage_away', 'cards_yellow_16-30_percentage_away',
    'cards_yellow_31-45_percentage_away', 'cards_yellow_46-60_percentage_away',
    'cards_yellow_61-75_percentage_away', 'cards_yellow_76-90_percentage_away',
    'cards_yellow_91-105_percentage_away',
    'cards_yellow_106-120_percentage_away', 'cards_red_0-15_percentage_away',
    'cards_red_16-30_percentage_away', 'cards_red_31-45_percentage_away',
    'cards_red_46-60_percentage_away', 'cards_red_61-75_percentage_away',
    'cards_red_76-90_percentage_away', 'cards_red_91-105_percentage_away',
    'cards_red_106-120_percentage_away'
]

for x in perc_columns:
    combined_data = transform_perc(combined_data, x)

Not a string!
Not a string!


In [94]:
columns_processed = [
    'goals_home', 'goals_away', 'ht_Shots_on_Goal', 'ht_Shots_off_Goal',
    'ht_Total_Shots', 'ht_Blocked_Shots', 'ht_Shots_insidebox',
    'ht_Shots_outsidebox', 'ht_Fouls', 'ht_Corner_Kicks', 'ht_Offsides',
    'ht_Ball_Possession', 'ht_Yellow_Cards', 'ht_Red_Cards',
    'ht_Goalkeeper_Saves', 'ht_Total_passes', 'ht_Passes_accurate',
    'ht_Passes_%', 'at_Shots_on_Goal', 'at_Shots_off_Goal', 'at_Total_Shots',
    'at_Blocked_Shots', 'at_Shots_insidebox', 'at_Shots_outsidebox',
    'at_Fouls', 'at_Corner_Kicks', 'at_Offsides', 'at_Ball_Possession',
    'at_Yellow_Cards', 'at_Red_Cards', 'at_Goalkeeper_Saves',
    'at_Total_passes', 'at_Passes_accurate', 'at_Passes_%',
    'fixtures_wins_home_home', 'fixtures_wins_away_home',
    'fixtures_draws_home_home', 'fixtures_draws_away_home',
    'fixtures_loses_home_home', 'fixtures_loses_away_home',
    'goals_for_total_home_home', 'goals_for_total_away_home',
    'goals_for_total_total_home', 'goals_for_average_home_home',
    'goals_for_average_away_home', 'goals_for_average_total_home',
    'goals_for_minute_0-15_total_home',
    'goals_for_minute_0-15_percentage_home',
    'goals_for_minute_16-30_total_home',
    'goals_for_minute_16-30_percentage_home',
    'goals_for_minute_31-45_total_home',
    'goals_for_minute_31-45_percentage_home',
    'goals_for_minute_46-60_total_home',
    'goals_for_minute_46-60_percentage_home',
    'goals_for_minute_61-75_total_home',
    'goals_for_minute_61-75_percentage_home',
    'goals_for_minute_76-90_total_home',
    'goals_for_minute_76-90_percentage_home',
    'goals_for_minute_91-105_total_home',
    'goals_for_minute_91-105_percentage_home',
    'goals_for_minute_106-120_total_home',
    'goals_for_minute_106-120_percentage_home',
    'goals_against_total_home_home', 'goals_against_total_away_home',
    'goals_against_total_total_home', 'biggest_streak_wins_home',
    'biggest_streak_draws_home', 'biggest_streak_loses_home',
    'biggest_wins_home_home', 'biggest_wins_away_home',
    'biggest_loses_home_home', 'biggest_loses_away_home',
    'biggest_goals_for_home_home', 'biggest_goals_for_away_home',
    'biggest_goals_against_home_home', 'biggest_goals_against_away_home',
    'clean_sheet_home_home', 'clean_sheet_away_home', 'clean_sheet_total_home',
    'failed_to_score_home_home', 'failed_to_score_away_home',
    'failed_to_score_total_home', 'penalty_scored_total_home',
    'penalty_scored_percentage_home', 'penalty_missed_total_home',
    'penalty_missed_percentage_home', 'cards_yellow_0-15_total_home',
    'cards_yellow_16-30_total_home', 'cards_yellow_31-45_total_home',
    'cards_yellow_46-60_total_home', 'cards_yellow_61-75_total_home',
    'cards_yellow_76-90_total_home', 'cards_yellow_91-105_total_home',
    'cards_yellow_106-120_total_home', 'cards_yellow_0-15_percentage_home',
    'cards_yellow_16-30_percentage_home', 'cards_yellow_31-45_percentage_home',
    'cards_yellow_46-60_percentage_home', 'cards_yellow_61-75_percentage_home',
    'cards_yellow_76-90_percentage_home',
    'cards_yellow_91-105_percentage_home',
    'cards_yellow_106-120_percentage_home', 'cards_red_0-15_total_home',
    'cards_red_16-30_total_home', 'cards_red_31-45_total_home',
    'cards_red_46-60_total_home', 'cards_red_61-75_total_home',
    'cards_red_76-90_total_home', 'cards_red_91-105_total_home',
    'cards_red_106-120_total_home', 'cards_red_0-15_percentage_home',
    'cards_red_16-30_percentage_home', 'cards_red_31-45_percentage_home',
    'cards_red_46-60_percentage_home', 'cards_red_61-75_percentage_home',
    'cards_red_76-90_percentage_home', 'cards_red_91-105_percentage_home',
    'cards_red_106-120_percentage_home', 'fixtures_wins_home_away',
    'fixtures_wins_away_away', 'fixtures_draws_home_away',
    'fixtures_draws_away_away', 'fixtures_loses_home_away',
    'fixtures_loses_away_away', 'goals_for_total_home_away',
    'goals_for_total_away_away', 'goals_for_total_total_away',
    'goals_for_average_home_away', 'goals_for_average_away_away',
    'goals_for_average_total_away', 'goals_for_minute_0-15_total_away',
    'goals_for_minute_0-15_percentage_away',
    'goals_for_minute_16-30_total_away',
    'goals_for_minute_16-30_percentage_away',
    'goals_for_minute_31-45_total_away',
    'goals_for_minute_31-45_percentage_away',
    'goals_for_minute_46-60_total_away',
    'goals_for_minute_46-60_percentage_away',
    'goals_for_minute_61-75_total_away',
    'goals_for_minute_61-75_percentage_away',
    'goals_for_minute_76-90_total_away',
    'goals_for_minute_76-90_percentage_away',
    'goals_for_minute_91-105_total_away',
    'goals_for_minute_91-105_percentage_away',
    'goals_for_minute_106-120_total_away',
    'goals_for_minute_106-120_percentage_away',
    'goals_against_total_home_away', 'goals_against_total_away_away',
    'goals_against_total_total_away', 'biggest_streak_wins_away',
    'biggest_streak_draws_away', 'biggest_streak_loses_away',
    'biggest_wins_home_away', 'biggest_wins_away_away',
    'biggest_loses_home_away', 'biggest_loses_away_away',
    'biggest_goals_for_home_away', 'biggest_goals_for_away_away',
    'biggest_goals_against_home_away', 'biggest_goals_against_away_away',
    'clean_sheet_home_away', 'clean_sheet_away_away', 'clean_sheet_total_away',
    'failed_to_score_home_away', 'failed_to_score_away_away',
    'failed_to_score_total_away', 'penalty_scored_total_away',
    'penalty_scored_percentage_away', 'penalty_missed_total_away',
    'penalty_missed_percentage_away', 'cards_yellow_0-15_total_away',
    'cards_yellow_16-30_total_away', 'cards_yellow_31-45_total_away',
    'cards_yellow_46-60_total_away', 'cards_yellow_61-75_total_away',
    'cards_yellow_76-90_total_away', 'cards_yellow_91-105_total_away',
    'cards_yellow_106-120_total_away', 'cards_yellow_0-15_percentage_away',
    'cards_yellow_16-30_percentage_away', 'cards_yellow_31-45_percentage_away',
    'cards_yellow_46-60_percentage_away', 'cards_yellow_61-75_percentage_away',
    'cards_yellow_76-90_percentage_away',
    'cards_yellow_91-105_percentage_away',
    'cards_yellow_106-120_percentage_away', 'cards_red_0-15_total_away',
    'cards_red_16-30_total_away', 'cards_red_31-45_total_away',
    'cards_red_46-60_total_away', 'cards_red_61-75_total_away',
    'cards_red_76-90_total_away', 'cards_red_91-105_total_away',
    'cards_red_106-120_total_away', 'cards_red_0-15_percentage_away',
    'cards_red_16-30_percentage_away', 'cards_red_31-45_percentage_away',
    'cards_red_46-60_percentage_away', 'cards_red_61-75_percentage_away',
    'cards_red_76-90_percentage_away', 'cards_red_91-105_percentage_away',
    'cards_red_106-120_percentage_away'
]

In [95]:
# Extract numbers
combined_data["biggest_wins_home_home"] = combined_data[
    "biggest_wins_home_home"].str.split("-").str[0]
combined_data["biggest_wins_away_home"] = combined_data[
    "biggest_wins_away_home"].str.split("-").str[1]

combined_data["biggest_loses_home_home"] = combined_data[
    "biggest_loses_home_home"].str.split("-").str[0]
combined_data["biggest_loses_away_home"] = combined_data[
    "biggest_loses_away_home"].str.split("-").str[1]

combined_data["biggest_wins_home_away"] = combined_data[
    "biggest_wins_home_away"].str.split("-").str[0]
combined_data["biggest_wins_away_away"] = combined_data[
    "biggest_wins_away_away"].str.split("-").str[1]

combined_data["biggest_loses_home_away"] = combined_data[
    "biggest_loses_home_away"].str.split("-").str[0]
combined_data["biggest_loses_away_away"] = combined_data[
    "biggest_loses_away_away"].str.split("-").str[1]


# Make diffs
combined_data["biggest_wins_home_home_diff"] = combined_data[
    "biggest_wins_home_home"].str.split("-").str[0] - combined_data["biggest_wins_home_home"].str.split(
            "-").str[1]
combined_data["biggest_wins_away_home_diff"] = combined_data[
    "biggest_wins_away_home"].str.split("-").str[1] - combined_data["biggest_wins_away_home"].str.split(
            "-").str[0]
combined_data["biggest_loses_home_home_diff"] = combined_data[
    "biggest_loses_home_home"].str.split("-").str[0] - combined_data["biggest_loses_home_home"].str.split(
            "-").str[1]
combined_data["biggest_loses_away_home_diff"] = combined_data[
    "biggest_loses_away_home"].str.split("-").str[1] - combined_data["biggest_loses_away_home"].str.split(
            "-").str[0]
combined_data["biggest_wins_home_away_diff"] = combined_data[
    "biggest_wins_home_away"].str.split("-").str[0] - combined_data["biggest_wins_home_away"].str.split(
            "-").str[1]
combined_data["biggest_wins_away_away_diff"] = combined_data[
    "biggest_wins_away_away"].str.split("-").str[1] - combined_data["biggest_wins_away_away"].str.split(
            "-").str[0]
combined_data["biggest_loses_home_away_diff"] = combined_data[
    "biggest_loses_home_away"].str.split("-").str[0] - combined_data["biggest_loses_home_away"].str.split(
            "-").str[1]
combined_data["biggest_loses_away_away_diff"] = combined_data[
    "biggest_loses_away_away"].str.split("-").str[1] - combined_data["biggest_loses_home_away"].str.split(
            "-").str[0]

# Change type
combined_data["biggest_wins_home_home"] = pd.to_numeric(combined_data[
    "biggest_wins_home_home"])
combined_data["biggest_wins_away_home"] = pd.to_numeric(combined_data[
    "biggest_wins_away_home"])

combined_data["biggest_loses_home_home"] = pd.to_numeric(combined_data[
    "biggest_loses_home_home"])
combined_data["biggest_loses_away_home"] = pd.to_numeric(combined_data[
    "biggest_loses_away_home"])

combined_data["biggest_wins_home_away"] = pd.to_numeric(combined_data[
    "biggest_wins_home_away"])
combined_data["biggest_wins_away_away"] = pd.to_numeric(combined_data[
    "biggest_wins_away_away"])

combined_data["biggest_loses_home_away"] = pd.to_numeric(combined_data[
    "biggest_loses_home_away"])
combined_data["biggest_loses_away_away"] = pd.to_numeric(combined_data[
    "biggest_loses_away_away"])

# Fillna with -1 
combined_data = combined_data.fillna(-1)
print(combined_data.shape)

(9158, 245)


## Time based features

In [96]:
# Sort values so we can get shifts
combined_data = combined_data.sort_values(["home_team_name", "date"])

# Create year variable
combined_data["year"] = pd.to_datetime(combined_data.date).dt.year

In [97]:
#combined_data_copy = combined_data.copy()
#combined_data = combined_data_copy

In [98]:
def create_lags(df, categories, target, lags=[1, 2, 3, 4, 5]):
    """Creates lagged values of a variable and category"""
    col_name_start = target + "_" + categories + "_"
    for lag in lags:
        # Group by and shift
        df[col_name_start + str(lag)] = df.\
            groupby([categories],
                    as_index=False)[target].\
            shift(lag).\
            astype(np.float16)

    columns_returned = [
        col_name_start + "1", col_name_start + "2", col_name_start + "3",
        col_name_start + "4", col_name_start + "5"
    ]

    return df[columns_returned]


# Run through lag creator, for each hiearchy team & league
# for hierarchy in ["home_team_name", "away_team_name", "league_name"]:
#    for var in columns_processed:
#        combined_data = create_lags(combined_data, hierarchy, var)

# Issues is that this creates multiple objects
for hierarchy in tqdm(["home_team_name", "away_team_name", "league_name"]):
    results = [
        Parallel(n_jobs=5)(delayed(create_lags)(combined_data, hierarchy, var)
                           for var in columns_processed)
    ]
    lagged_variables = pd.concat(results[0], axis=1)
    combined_data = pd.concat([combined_data, lagged_variables], axis = 1)


print(combined_data.shape)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:27<00:00,  9.08s/it]

(9158, 3276)


In [99]:
def windower(df,
             categories,
             target,
             windower="rolling",
             window_function="mean",
             rolling_length=2):
    """Performs expanding and rolling windows"""
    df[target + "_" + categories + "_" + windower + "_" +
       window_function] = None  # Empty variable with name

    # Notice logic structure: windower -> window_function
    # Each function follow the same logic for the above created variable,
    # take the lag 1 value (avoid data leakge) and perform rolling or expanding window.
    if windower == "rolling":
        if window_function == "mean":
            df[target + "_" + categories + "_" + windower + "_" + window_function + "_" + str(rolling_length)] = df.\
                groupby([categories])[target + "_" + categories + "_1"].\
                transform(lambda x: x.rolling(
                    rolling_length).mean()).astype(np.float16)

        if window_function == "max":
            df[target + "_" + categories + "_" + windower + "_" + window_function + "_" + str(rolling_length)] = df.\
                groupby([categories])[target + "_" + categories + "_1"].\
                transform(lambda x: x.rolling(
                    rolling_length).max()).astype(np.float16)
        if window_function == "min":
            df[target + "_" + categories + "_" + windower + "_" + window_function + "_" + str(rolling_length)] = df.\
                groupby([categories])[target + "_" + categories + "_1"].\
                transform(lambda x: x.rolling(
                    rolling_length).min()).astype(np.float16)
        if window_function == "std":
            df[target + "_" + categories + "_" + windower + "_" + window_function + "_" + str(rolling_length)] = df.\
                groupby([categories])[target + "_" + categories + "_1"].\
                transform(lambda x: x.rolling(
                    rolling_length).std()).astype(np.float16)

    if windower == "expanding":
        if window_function == "mean":
            df[target + "_" + categories + "_" + windower + "_" + window_function] = df.\
                groupby([categories])[target + "_" + categories + "_1"].\
                transform(lambda x: x.expanding(2).mean()).astype(np.float16)
        if window_function == "max":
            df[target + "_" + categories + "_" + windower + "_" + window_function] = df.\
                groupby([categories])[target + "_" + categories + "_1"].\
                transform(lambda x: x.expanding(2).max()).astype(np.float16)
        if window_function == "min":
            df[target + "_" + categories + "_" + windower + "_" + window_function] = df.\
                groupby([categories])[target + "_" + categories + "_1"].\
                transform(lambda x: x.expanding(2).min()).astype(np.float16)
        if window_function == "std":
            df[target + "_" + categories + "_" + windower + "_" + window_function] = df.\
                groupby([categories])[target + "_" + categories + "_1"].\
                transform(lambda x: x.expanding(2).std()).astype(np.float16)

    return df

def windower_meta(df,
                  hierarchy,
                  var,
                  window_types=["rolling", "expanding"],
                  window_functions=["mean", "max", "min", "std"]):
    """Used for orchestrating multiple windowing functions."""

    variables = df.shape[1]

    for window_type in window_types:
        for window_function in window_functions:
            if window_type == "rolling":
                for length in [2, 3, 4, 5, 6, 8, 10, 15, 20]:
                    df = windower(df, hierarchy, var, window_type,
                                  window_function, length)

            else:
                df = windower(df, hierarchy, var, window_type, window_function)
    return df.iloc[:, variables:]

## Go through each hierarchy and create lags
#for hierarchy in ["home_team_name", "away_team_name", "league_name"]:
#    for var in tqdm(columns_processed):
#        for window_type in ["rolling", "expanding"]:  # window type
#            for window_function in ["mean", "max", "min",
#                                    "std"]:  # function types
#                if window_type == "rolling":
#                    temp_results = [Parallel(n_jobs=9)(
#                        delayed(windower)(combined_data, hierarchy, var,
#                                          window_type, window_function, length)
#                        for length in [2, 3, 4, 5, 6, 8, 10, 15, 20])]
#                    
#                    temp_results = pd.concat(temp_results[0], axis = 1)
#                    combined_data = pd.concat([combined_data,temp_results], axis=1)
#                    
#                else:
#                    temp_results = windower(combined_data, hierarchy, var,
#                                             window_type, window_function)
#                    combined_data = pd.concat([combined_data,temp_results], axis=1)
#print(results.shape)

In [100]:
# Go through each hierarchy and create lags
for hierarchy in ["home_team_name", "away_team_name", "league_name"]:
    temp_data = [
        Parallel(n_jobs=35)(
            delayed(windower_meta)(combined_data, hierarchy, var)
            for var in tqdm(columns_processed))
    ]
    moving_averages = pd.concat(temp_data[0], axis=1)
    combined_data = pd.concat([combined_data, moving_averages], axis=1)

 52%|████████████████████████████████████████████████▊                                             | 105/202 [00:58<00:49,  1.96it/s]/Users/august/opt/miniconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 202/202 [02:57<00:00,  1.14it/s]


In [101]:
# Trend
def trender(df, target, category, trend_length=[2, 4, 6]):
    """Create trend columns, using expanding mean and rolling."""
    variables = df.shape[1]
    df_copy = df.copy()
    for trend in trend_length:
        df_copy[target + "_" + str(trend) + "_trend_mean"] = df_copy[target + "_" + category + "_rolling_mean_" + str(trend)] - \
            df_copy[target + "_" + category + "_expanding_mean"]
    return df_copy.iloc[:, variables:]


# Go through each iherarchy and create trends
for hierarchy in ["home_team_name",
        "away_team_name", "league_name"
]:
    temp_results = [
        Parallel(n_jobs=35)(delayed(trender)(combined_data, var, hierarchy)
                            for var in tqdm(columns_processed))
    ]
    trends = pd.concat(temp_results[0], axis=1)
    combined_data = pd.concat([combined_data, trends], axis=1)

print(combined_data.shape)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 202/202 [03:58<00:00,  1.18s/it]


(9158, 30546)


## Finalization

In [ ]:
# Fill na's
combined_data = combined_data.replace(-1, np.nan)
combined_data = combined_data.dropna(how="all", axis=1)
combined_data = combined_data.fillna(-1)
one_hot_data.reset_index(inplace=True, drop=True)
combined_data.reset_index(inplace=True, drop=True)

combined_data = pd.concat([combined_data, one_hot_data], axis=1)
print(combined_data.shape)

In [112]:
combined_data.head()

,id,date,status,league_id,league_name,home_team_id,home_team_name,away_team_id,away_team_name,goals_home,...,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21
0,209848,2019-01-26 19:45:00+00:00,FT,45,FA Cup,1333,AFC Wimbledon,48,West Ham,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,209825,2019-02-16 15:00:00+00:00,FT,45,FA Cup,1333,AFC Wimbledon,58,Millwall,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,648794,2020-11-29 13:30:00+00:00,FT,45,FA Cup,1333,AFC Wimbledon,1362,Crawley Town,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,648794,2020-11-29 13:30:00+00:00,FT,45,FA Cup,1333,AFC Wimbledon,1362,Crawley Town,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,648794,2020-11-29 13:30:00+00:00,FT,45,FA Cup,1333,AFC Wimbledon,1362,Crawley Town,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
combined_data.to_csv(os.getcwd() + "/data/processed_data_2020_prem_champ.csv", index=None)
